In [1]:
from datasets import *

In [21]:
"""
This code implements functionalities required for computing the NT Kernel for multi-layer
fully-connected neural networks. The computed kernels are saved to the disk. 

The code is written for Python 3.6. 

Inputs: 
	noise_id: The index of the noise intensity: valid range 0 to 14.
	num_layers: The number of layers: valid range {2, 3, 4}.
"""

from __future__ import print_function
import math
import os 
import sys
import time
import numpy as np

from jax import random
from neural_tangents import stax

num_layers = 2
dataset_name = "CIFAR2"
labels = [0, 3]
ratio = 1
X, Y, Xtest, Ytest = load_dataset(dataset_name, labels=labels, ratio=1.0, grayscale=True)
X = X.reshape(X.shape[0], -1)
Xtest = Xtest.reshape(Xtest.shape[0], -1)


if num_layers == 2:
	init_fn, apply_fn, kernel_fn = stax.serial(stax.Dense(1024), stax.Relu(), stax.Dense(1))
elif num_layers == 3:
	init_fn, apply_fn, kernel_fn = stax.serial(stax.Dense(1024), stax.Relu(), stax.Dense(1024), stax.Relu(), stax.Dense(1))
elif num_layers == 4:
	init_fn, apply_fn, kernel_fn = stax.serial(stax.Dense(1024), stax.Relu(), stax.Dense(1024), stax.Relu(), stax.Dense(1024), stax.Relu(), stax.Dense(1))
else:
	raise Exception('Non-valid Kernel')

n = X.shape[0]
print(f"Data points: {n}")
kernel = np.zeros((n, n), dtype=np.float32)
batch_size = 10
m = n / batch_size
m = int(m)
# To avoid memory overflow, for training data, we fill the kernel matrix block by block
for i in range(batch_size):
    for j in range(batch_size):
        print('%d and %d'%(i, j))
        x1 = X[i * m:(i + 1) * m, :].reshape(m, -1)
        x2 = X[j * m:(j + 1) * m, :].reshape(m, -1)
        kernel[i * m:(i + 1) * m, j * m:(j + 1) * m] = kernel_fn(x1, x2, 'ntk')
print(kernel.shape)
directory = './NTK_Kernels/'
directory += f"{dataset_name}/"+f"labels_{labels[0]}_{labels[1]}_ratio_{ratio}/"
if not os.path.exists(directory):
    os.makedirs(directory)
file_name = 'Train_NTK_layers_%d.npy'%(num_layers)
np.save(directory + file_name, kernel)

file_name = 'Test_NTK_layers_%d.npy'%(num_layers)
kernel = kernel_fn(Xtest, X, 'ntk')
np.save(directory + file_name, kernel)

Data points: 10000
0 and 0
0 and 1
0 and 2
0 and 3
0 and 4
0 and 5
0 and 6
0 and 7
0 and 8
0 and 9
1 and 0
1 and 1
1 and 2
1 and 3
1 and 4
1 and 5
1 and 6
1 and 7
1 and 8
1 and 9
2 and 0
2 and 1
2 and 2
2 and 3
2 and 4
2 and 5
2 and 6
2 and 7
2 and 8
2 and 9
3 and 0
3 and 1
3 and 2
3 and 3
3 and 4
3 and 5
3 and 6
3 and 7
3 and 8
3 and 9
4 and 0
4 and 1
4 and 2
4 and 3
4 and 4
4 and 5
4 and 6
4 and 7
4 and 8
4 and 9
5 and 0
5 and 1
5 and 2
5 and 3
5 and 4
5 and 5
5 and 6
5 and 7
5 and 8
5 and 9
6 and 0
6 and 1
6 and 2
6 and 3
6 and 4
6 and 5
6 and 6
6 and 7
6 and 8
6 and 9
7 and 0
7 and 1
7 and 2
7 and 3
7 and 4
7 and 5
7 and 6
7 and 7
7 and 8
7 and 9
8 and 0
8 and 1
8 and 2
8 and 3
8 and 4
8 and 5
8 and 6
8 and 7
8 and 8
8 and 9
9 and 0
9 and 1
9 and 2
9 and 3
9 and 4
9 and 5
9 and 6
9 and 7
9 and 8
9 and 9
(10000, 10000)


In [25]:
from pvr_datasets import *

In [28]:
X, Y = create_boolean_pvr(pointer_label, 3)
torch.save((X, Y), "data/pvr/pointer_3")
X, Y = torch.load("data/pvr/pointer_3")

In [ ]:
def perform_iterations(iterations, dataset_name, kernel, reg_values, datasets, **kwargs):
    IKM = IterativeKernelModel(dataset_name, kernel=kernel, datasets=datasets)
    results = {}
    for it in range(1, iterations+1):
        print(f"iteration: {it}")
        if it>1:
            yhat, preds = normalize(yhat, preds)
            IKM.D_train = np.diag(yhat.flatten(), k=0)
            IKM.D_test = np.diag(preds.flatten(), k=0)
            IKM.make_kernel_matrices(ind=it-1, kernel=kernel)
            weights = list(np.ones(it)/(it))
            IKM.combine_kernels(weights)
        else:
            weights = list(np.ones(it))
        yhat, preds, res = IKM.solve(reg)
        res["weights"], res["dataset name"] = weights, dataset_name
        results[f"iter {it}"] = res
    return results

In [ ]:
dataset_name = "CIFAR2"
kernel = "rbf"
labels = [0, 3]
ratio = 1.0

datasets = load_dataset(dataset_name, ratio=ratio, labels=labels)
results = {}
actions = ["1"]
for reg_1 in 10**np.array([ -1.0, -0.5, 0, 0.5, 1]):
    
    IKM = IterativeKernelModel(dataset_name, kernel=kernel, datasets=datasets)
    IKM.normalize_kernels()
    actions.append(str(reg_1))
    yhat, preds, res = IKM.solve(reg)
    # results[f"var: {var}, reg: {reg}"] = res
    results[reg] = res
    
results[var] = var_results

In [ ]:
setups = {}
labels_list = [[3, 0]] #, [3, 5]
for labels in labels_list:
    for ratio in [1]: #0.1, 0.25, 0.5, 0.8, 
        setups[f"labels {labels}, {ratio*100}% data"] = {"dataset_name": "CIFAR2",
                                                         "iterations": 3,
                                                         "reg": 2,
                                                         "labels": labels,
                                                         "ratio": ratio}
        

In [ ]:
for case, setup in setups.items():
    print(f"---------------- {case} ----------------")
    dataset_name = setup["dataset_name"]
    datasets = load_dataset(dataset_name, ratio=setup["ratio"], labels=setup["labels"])
    reg, iterations = setup["reg"], setup["iterations"]
    res = {}
    for kernel in ["rbf"]: #, "ntk"
        print(f"------ {kernel} -----")
        res[kernel] = perform_iterations(iterations, dataset_name, kernel, reg, datasets)
    # print("------ NN -----")
    # res["2-layer NN 1000"] = train_test_NN(datasets, epochs=20)
    # results[case] = res
    # torch.save(results, "./res.tr")

In [ ]:
categories = "Cat, Airplane"
labels = [3, 0]
case_summary = {}
for kernel in ["rbf", "ntk"]:
    for it in range(1, 4):
        acc_list = []
        for ratio in [0.1, 0.25, 0.5, 0.8, 1]:
            res = results[f"labels {labels}, {ratio*100}% data"]
            res = res[kernel]
            res = res[f"iter {it}"]
            acc_list.append(res["Test accuracy"])
        case_summary[kernel + f"-{it}"] = acc_list
acc_list = []
for ratio in [0.1, 0.25, 0.5, 0.8, 1]:
    res = results[f"labels {labels}, {ratio*100}% data"]
    res = res["2-layer NN 1000"]
    acc_list.append(res["Test accuracy"])
case_summary["NN"] = acc_list
accuracy_summary[categories] = case_summary

In [ ]:
list(map(lambda x: f"{int(100*x)}%", [0.1, 0.25, 0.5, 0.8, 1]))

In [1]:
import torch
import pandas as pd

/home/talaei/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
logs = torch.load("./results/CIFAR2/0_3/level_2_0.25")

In [17]:
len(logs.keys())

972

In [15]:
32 * 36

1152

In [18]:
df = pd.DataFrame(logs).transpose()

In [19]:
display(df)

,reg,Train error,Test error,Train accuracy,Test accuracy
/1: -> rbf-> reg 0.100,0.100000,0.013948,0.520102,0.9996,0.8275
"/1: -> rbf-> reg 0.100 /2: -> rbf-> g identity-> mix (0.75, 0.25)-> reg 0.100",0.100000,0.000302,0.520057,1.0000,0.8250
"/1: -> rbf-> reg 0.100 /2: -> rbf-> g identity-> mix (0.75, 0.25)-> reg 1.000",1.000000,0.002781,0.520792,1.0000,0.8275
"/1: -> rbf-> reg 0.100 /2: -> rbf-> g identity-> mix (0.75, 0.25)-> reg 3.162",3.162277,0.008794,0.522705,1.0000,0.8270
"/1: -> rbf-> reg 0.100 /2: -> rbf-> g identity-> mix (0.75, 0.25)-> reg 10.000",9.999999,0.033819,0.530358,0.9996,0.8285
...,...,...,...,...,...
"/1: -> linear-> reg 100.000 /2: -> linear-> g normalize-> mix (0.0, 1.0)-> reg 0.100",0.100000,0.001048,0.514131,0.9996,0.8275
"/1: -> linear-> reg 100.000 /2: -> linear-> g normalize-> mix (0.0, 1.0)-> reg 1.000",1.000000,0.003421,0.516325,0.9996,0.8275
"/1: -> linear-> reg 100.000 /2: -> linear-> g normalize-> mix (0.0, 1.0)-> reg 3.162",3.162278,0.005558,0.518580,0.9996,0.8275
"/1: -> linear-> reg 100.000 /2: -> linear-> g normalize-> mix (0.0, 1.0)-> reg 10.000",10.000000,0.009461,0.521748,0.9996,0.8275


In [5]:
import gspread

gc = gspread.service_account(filename='iterative-learning-methods-4a3e9c5a47ae.json')

# sht1 = gc.open_by_key('1vF_0iuL4E_sfiLjcL1dePgbHShk5Qj3R0gYJeCkXpCk')
sht2 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1vF_0iuL4E_sfiLjcL1dePgbHShk5Qj3R0gYJeCkXpCk/edit#gid=0')
# sh = gc.open("Evolving Kernels")

# print(sh.sheet1.get('A1'))

KeyboardInterrupt: 